In [95]:
!python -m pip install zipcodes

In [1]:
import os
import sys
sys.path.insert(0, "../")

In [2]:
import pandas as pd
import numpy as np
from data_processing.data_cleaning_utils import clean_accepted_df
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

## Loading LendingClub Data

It is expected that the accepted and rejected loan datasets live in a subdirectory called ```data``` in the root directory.

In [189]:
DATASET_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data/')
ACCEPTED_LOANS_FN = "Lending_Club_Accepted_2014_2018.csv"
REJECTED_LOANS_FN = "Lending_Club_Rejected_2014_2018.csv"

accepted = pd.read_csv(DATASET_PATH + ACCEPTED_LOANS_FN)
rejected = pd.read_csv(DATASET_PATH + REJECTED_LOANS_FN)

/home/acowlagi/.conda/envs/torch_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Exploring the Accepted Loans Dataset

In [190]:
accepted.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,Aug-2003,675.0,679.0,1.0,30.0,NaN,7.0,0.0,2765.0,29.7,13.0,w,0.00,0.00,4421.723917,4421.72,3600.00,821.72,0.0,0.0,0.0,Jan-2019,122.67,NaN,Mar-2019,564.0,560.0,0.0,30.0,1.0,Individual,NaN,NaN,NaN,0.0,722.0,144904.0,2.0,2.0,0.0,1.0,21.0,4981.0,36.0,3.0,3.0,722.0,34.0,9300.0,3.0,1.0,4.0,4.0,20701.0,1506.0,37.2,0.0,0.0,148.0,128.0,3.0,3.0,1.0,4.0,69.0,4.0,69.0,2.0,2.0,4.0,2.0,5.0,3.0,4.0,9.0,4.0,7.0,0.0,0.0,0.0,3.0,76.9,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,small_business,Business,577xx,SD,16.06,1.0,Dec-1999,715.0,719.0,4.0,6.0,NaN,22.0,0.0,21470.0,19.2,38.0,w,0.00,0.00,25679.660000,25679.66,24700.00,979.66,0.0,0.0,0.0,Jun-2016,926.35,NaN,Mar-2019,699.0,695.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,204396.0,1.0,1.0,0.0,1.0,19.0,18005.0,73.0,2.0,3.0,6472.0,29.0,111800.0,0.0,0.0,6.0,4.0,9733.0,57830.0,27.1,0.0,0.0,113.0,192.0,2.0,2.0,4.0,2.0,NaN,0.0,6.0,0.0,5.0,5.0,13.0,17.0,6.0,20.0,27.0,5.0,22.0,0.0,0.0,0.0,2.0,97.4,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,NaN,NaN,NaN,NaN,NaN,N

### Subsampling By Loan Amount

For the sake of initial exploration and establishing the cleaning methodology, we sample from the accepted loans data to construct a representative subset of the data. We do this by sampling from a discrete approximation of the distribution of ```loan_amnt``` category to apply a 50x reduction in the dataset size.

In [3]:
TEST_PATH = os.path.join(os.path.dirname(os.getcwd()), 'data_processing/test_files/')
SAMPLE_BY_LOAN_FN = "sample_by_loan_amt.csv"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 160)

In [4]:
sampled_by_loan = pd.read_csv(TEST_PATH + SAMPLE_BY_LOAN_FN)
sampled_by_loan.head()

/home/acowlagi/.conda/envs/torch_py39/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (19,59,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68533595,NaN,12000.0,12000.0,12000.0,36 months,10.78,391.62,B,B4,Public Affairs Specialist,1 year,RENT,98000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,993xx,WA,24.04,0.0,Feb-2002,690.0,694.0,0.0,33.0,61.0,15.0,1.0,20462.0,62.8,39.0,w,0.0,0.0,13349.665589,13349.67,12000.0,1349.67,0.0,0.00,0.0000,Apr-2017,7884.96,NaN,Jun-2017,754.0,750.0,0.0,69.0,1.0,Individual,NaN,NaN,NaN,0.0,5930.0,374585.0,0.0,4.0,1.0,3.0,9.0,82819.0,66.0,1.0,2.0,12597.0,65.0,32600.0,1.0,1.0,0.0,5.0,26756.0,3884.0,83.9,0.0,0.0,147.0,166.0,9.0,9.0,8.0,9.0,69.0,15.0,69.0,4.0,3.0,5.0,3.0,6.0,17.0,8.0,14.0,5.0,15.0,0.0,0.0,0.0,2.0,87.2,66.7,0.0,0.0,478544.0,103281.0,24100.0,120544.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68495092,NaN,8650.0,8650.0,8650.0,36 months,19.89,320.99,E,E3,Program Coordinator,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,462xx,IN,25.49,0.0,Mar-2005,675.0,679.0,4.0,NaN,30.0,18.0,1.0,9568.0,46.0,19.0,w,0.0,0.0,9190.490000,9190.49,8650.0,540.49,0.0,0.00,0.0000,May-2016,8251.42,NaN,Jun-2016,639.0,635.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,18926.0,6.0,1.0,0.0,1.0,19.0,9358.0,51.0,12.0,16.0,653.0,46.0,20750.0,2.0,0.0,5.0,17.0,1051.0,1375.0,45.0,0.0,0.0,129.0,95.0,0.0,0.0,0.0,8.0,NaN,0.0,NaN,0.0,2.0,17.0,2.0,2.0,2.0,17.0,17.0,13.0,18.0,NaN,0.0,0.0,12.0,100.0,50.0,1.0,0.0,38998.

We now explore the structure of various features of initial interest. We drop the remaining columns for this section. In general, we have extracted features that are known prior to/immediately after loan origination, rather than at the end of the lifetime of the loan. We also include the loan status. 

In [5]:
cat_vars_of_interest = ["term", "grade", "sub_grade", "emp_title", "emp_length", "home_ownership", "verification_status", "purpose", "desc", "addr_state", 
    "initial_list_status", "policy_code", "application_type", "hardship_flag", "hardship_type", "hardship_reason",  "loan_status"
]
num_vars_of_interest = list(set([
    "loan_amnt", "funded_amnt", "funded_amnt_inv", "int_rate", "installment", "issue_d", "annual_inc", "dti", "fico_range_low", "fico_range_high", 
    "revol_bal", "revol_util", "open_acc", "zip_code", "delinq_2yrs", "inq_last_6mths", "total_acc", "mths_since_last_delinq", "mths_since_last_record", "mths_since_rcnt_il",
    "last_fico_range_low", "last_fico_range_high", "last_credit_pull_d", "open_il_12m", "open_il_24m", "total_bal_il", "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc", "all_util",
    "total_rev_hi_lim", "inq_fi", "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy", "bc_util", "num_accts_ever_120_pd",
    "num_actv_bc_tl", "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_sats", "num_il_tl", "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "tot_hi_cred_lim", 
    "pct_tl_nvr_dlq", "percent_bc_gt_75", "total_bal_ex_mort", "total_bc_limit","total_il_high_credit_limit", "mths_since_last_major_derog", "mths_since_recent_bc",
    "mths_since_recent_bc_dlq", "mths_since_recent_inq", "mths_since_recent_revol_delinq"
]))


In [6]:
sampled_by_loan = sampled_by_loan[num_vars_of_interest + cat_vars_of_interest]
sampled_by_loan.head()

,mths_since_rcnt_il,open_il_12m,loan_amnt,mths_since_recent_bc,total_bc_limit,total_il_high_credit_limit,pct_tl_nvr_dlq,total_cu_tl,mths_since_last_delinq,issue_d,mths_since_recent_bc_dlq,mths_since_recent_inq,fico_range_low,funded_amnt_inv,num_bc_tl,max_bal_bc,last_credit_pull_d,dti,total_bal_ex_mort,installment,num_op_rev_tl,percent_bc_gt_75,il_util,bc_open_to_buy,num_accts_ever_120_pd,zip_code,delinq_2yrs,open_rv_24m,mths_since_last_record,num_bc_sats,mths_since_recent_revol_delinq,inq_last_12m,acc_open_past_24mths,annual_inc,open_rv_12m,int_rate,last_fico_range_low,inq_fi,num_actv_rev_tl,num_rev_tl_bal_gt_0,open_acc,total_acc,num_il_tl,mths_since_last_major_derog,total_bal_il,bc_util,tot_hi_cred_lim,revol_util,num_actv_bc_tl,all_util,revol_bal,avg_cur_bal,funded_amnt,last_fico_range_high,open_il_24m,total_rev_hi_lim,num_sats,num_rev_accts,inq_last_6mths,fico_range_high,term,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,purpose,desc,addr_state,initial_list_status,policy_code,application_type,hardship_flag,hardship_type,hardship_reason,loan_status
0,9.0,1.0,12000.0,9.0,24100.0,120544.0,87.2,1.0,33.0,Dec-2015,69.0,15.0,690.0,12000.0,6.0,12597.0,Jun-2017,24.04,103281.0,391.62,8.0,66.7,66.0,3884.0,4.0,993xx,0.0,2.0,61.0,3.0,69.0,0.0,5.0,98000.0,1.0,10.78,750.0,1.0,5.0,5.0,15.0,39.0,17.0,69.0,82819.0,83.9,478544.0,62.8,3.0,65.0,20462.0,26756.0,12000.0,754.0,3.0,32600.0,15.0,14.0,0.0,694.0,36 months,B,B4,Public Affairs Specialist,1 year,RENT,Not Verified,debt_consolidation,NaN,WA,w,1.0,Individual,N,NaN,NaN,Fully Paid
1,19.0,0.0,8650.0,8.0,2750.0,18248.0,100.0,0.0,NaN,Dec-2015,NaN,0.0,675.0,8650.0,2.0,653.0,Jun-2016,25.49,18926.0,320.99,17.0,50.0,51.0,1375.0,0.0,462xx,0.0,16.0,30.0,2.0,NaN,5.0,17.0,55000.0,12.0,19.89,635.0,2.0,17.0,13.0,18.0,19.0,2.0,NaN,9358.0,45.0,38998.0,46.0,2.0,46.0,9568.0,1051.0,8650.0,639.0,1.0,20750.0,18.0,17.0,4.0,679.0,36 months,E,E3,Program Coordinator,8 years,RENT,Verified,debt_consolidation,NaN,IN,w,1.0,Individual,N,NaN,NaN,Fully Paid
2,13.0,0.0,25000.0,13.0,38400.0,82117.0,100.0,0.0,NaN,Dec-2015,NaN,0.0,745.0,25000.0,6.0,8937.0,Apr-2017,26.02,68056.0,777.55,5.0,20.0,58.0,17538.0,0.0,226xx,0.0,1.0,NaN,5.0,NaN,1.0,2.0,109000.0,0.0,7.49,720.0,1.0,3.0,3.0,9.0,19.0,7.0,NaN,47194.0,54.3,373572.0,54.3,3.0,57.0,20862.0,33976.0,25000.0,724.0,1.0,38400.0,9.0,9.0,1.0,749.0,36 months,A,A4,Sales Manager,10+ years,MORTGAGE,Not Verified,debt_consolidation,NaN,VA,w,1.0,Individual,N,NaN,NaN,Fully Paid
3,47.0,0.0,4225.0,26.0,2600.0,17522.0,66.7,0.0,18.0,Dec-2015,NaN,14.0,725.0,4225.0,3.0,367.0,Dec-2017,15.22,4888.0,146.16,5.0,0.0,22.0,2233.0,0.0,672xx,2.0,0.0,NaN,3.0,18.0,0.0,0.0,35000.0,0.0,14.85,0.0,0.0,2.0,2.0,6.0,6.0,1.0,NaN,3830.0,14.1,21822.0,24.6,1.0,22.0,1058.0,815.0,4225.0,499.0,0.0,4300.0,6.0,5.0,0.0,729.0,36 months,C,C5,mechanic,5 years,RENT,Source Verified,debt_consolidation,NaN,KS,w,1.0,Individual,N,NaN,NaN,Charged Off
4,2.0,1.0,1400.0,28.0,34500.0,55501.0,100.0,2.0,NaN,Dec-2015,NaN,18.0,700.0,1400.0,11.0,7386.0,Sep-2018,34.95,75258.0,47.10,12.0,75.0,67.0,7940.0,0.0,275xx,0.0,2.0,NaN,8.0,NaN,0.0,4.0,64000.0,0.0,12.88,700.0,0.0,11.0,11.0,17.0,24.0,3.0,NaN,37430.0,77.0,372109.0,67.2,7.0,67.0,37828.0,19159.0,1400.0,704.0,2.0,56300.0,17.0,17.0,0.0,704.0,36 months,C,C2,Logistics Manager,3 years,MORTGAGE,Not Verified,other,NaN,NC,w,1.0,Individual,N,NaN,NaN,Fully Paid


In [7]:
missing = sampled_by_loan.isna().sum()
missing /= sampled_by_loan.shape[0]
missing *=100
missing = missing.to_frame().rename(columns={0:'Percent Of Missing Values'})
missing

,Percent Of Missing Values
mths_since_rcnt_il,33.380927
open_il_12m,31.261385
loan_amnt,0.000000
mths_since_recent_bc,1.046231
total_bc_limit,0.000000
total_il_high_credit_limit,0.000000
pct_tl_nvr_dlq,0.000000
total_cu_tl,31.261385
mths_since_last_delinq,50.465265
issue_d,0.000000


We now make some plots to check the distribution of the different variables, so the cleaning stage is most productive

#### Numerical Features

In [8]:
numerical_features = sampled_by_loan.select_dtypes(include=['int64','float64']).columns.values
other_features = sampled_by_loan.select_dtypes(include=['object'])

numerical_features

array(['mths_since_rcnt_il', 'open_il_12m', 'loan_amnt',
       'mths_since_recent_bc', 'total_bc_limit',
       'total_il_high_credit_limit', 'pct_tl_nvr_dlq', 'total_cu_tl',
       'mths_since_last_delinq', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'fico_range_low', 'funded_amnt_inv',
       'num_bc_tl', 'max_bal_bc', 'dti', 'total_bal_ex_mort',
       'installment', 'num_op_rev_tl', 'percent_bc_gt_75', 'il_util',
       'bc_open_to_buy', 'num_accts_ever_120_pd', 'delinq_2yrs',
       'open_rv_24m', 'mths_since_last_record', 'num_bc_sats',
       'mths_since_recent_revol_delinq', 'inq_last_12m',
       'acc_open_past_24mths', 'annual_inc', 'open_rv_12m', 'int_rate',
       'last_fico_range_low', 'inq_fi', 'num_actv_rev_tl',
       'num_rev_tl_bal_gt_0', 'open_acc', 'total_acc', 'num_il_tl',
       'mths_since_last_major_derog', 'total_bal_il', 'bc_util',
       'tot_hi_cred_lim', 'revol_util', 'num_actv_bc_tl', 'all_util',
       'revol_bal', 'avg_cur_bal', 'funded_

In [9]:
print(len(numerical_features))
sampled_by_loan[numerical_features].describe()

58


,mths_since_rcnt_il,open_il_12m,loan_amnt,mths_since_recent_bc,total_bc_limit,total_il_high_credit_limit,pct_tl_nvr_dlq,total_cu_tl,mths_since_last_delinq,mths_since_recent_bc_dlq,mths_since_recent_inq,fico_range_low,funded_amnt_inv,num_bc_tl,max_bal_bc,dti,total_bal_ex_mort,installment,num_op_rev_tl,percent_bc_gt_75,il_util,bc_open_to_buy,num_accts_ever_120_pd,delinq_2yrs,open_rv_24m,mths_since_last_record,num_bc_sats,mths_since_recent_revol_delinq,inq_last_12m,acc_open_past_24mths,annual_inc,open_rv_12m,int_rate,last_fico_range_low,inq_fi,num_actv_rev_tl,num_rev_tl_bal_gt_0,open_acc,total_acc,num_il_tl,mths_since_last_major_derog,total_bal_il,bc_util,tot_hi_cred_lim,revol_util,num_actv_bc_tl,all_util,revol_bal,avg_cur_bal,funded_amnt,last_fico_range_high,open_il_24m,total_rev_hi_lim,num_sats,num_rev_accts,inq_last_6mths,fico_range_high,policy_code
count,27062.00000,27923.000000,40622.000000,40197.000000,40622.000000,40622.000000,40622.000000,27923.000000,20122.000000,9708.000000,36177.000000,40622.000000,40622.000000,40622.000000,27923.000000,40595.000000,40622.000000,40622.000000,40622.000000,40157.000000,23835.000000,40169.000000,40622.000000,40622.000000,27923.000000,6907.000000,40622.000000,13861.000000,27923.000000,40622.000000,4.062200e+04,27923.000000,40622.000000,40622.000000,27923.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,11100.000000,27923.000000,40149.000000,4.062200e+04,40595.000000,40622.000000,27919.000000,40622.000000,40621.000000,40622.000000,40622.000000,27923.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.0
mean,21.44350,0.678366,15186.276525,24.423937,23684.479272,44444.088376,93.989161,1.514307,34.494881,39.623094,6.989081,698.315568,15181.559223,7.661612,5825.924471,19.119201,51733.410763,448.875696,8.341662,41.097187,69.082987,11857.681620,0.518143,0.320885,2.806969,71.686695,4.809463,35.732848,2.048705,4.609670,7.832073e+04,1.311965,13.016267,676.527867,0.999749,5.663335,5.602211,11.783393,24.288169,8.474423,44.328649,35595.129499,56.870326,1.781516e+05,49.516155,3.684161,56.861456,16795.099035,13360.778317,15186.276525,688.375265,1.553952,35288.389050,11.739304,14.043548,0.550416,702.315716,1.0
std,26.39655,0.927429,9290.192340,31.971050,23931.433835,45442.334742,9.187799,2.722296,22.000156,22.953139,5.946621,33.073082,9288.291414,4.732256,5915.544238,14.708373,49814.550767,270.441946,4.843187,36.165645,23.882915,17454.408493,1.367721,0.886263,2.656030,25.550533,3.140370,22.488916,2.400769,3.223653,5.949613e+04,1.534661,4.874614,109.314856,1.481870,3.453474,3.344764,5.822492,12.215184,7.410487,21.665417,43941.903930,28.648460,1.785255e+05,24.750324,2.355986,20.998032,22423.222574,16231.061063,9290.192340,71.797607,1.567449,34974.412526,5.807124,8.229270,0.850023,33.073739,0.0
min,0.00000,0.000000,1000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,660.000000,750.000000,0.000000,0.000000,0.000000,0.000000,30.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.310000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,664.000000,1.0
25%,7.00000,0.000000,8000.000000,6.000000,8400.000000,15145.750000,91.000000,0.000000,16.000000,20.000000,2.000000,675.000000,8000.000000,4.000000,2274.000000,12.090000,21075.000000,252.880000,5.000000,0.000000,55.000000,1859.000000,0.000000,0.000000,1.000000,55.000000,3.000000,17.000000,0.000000,2.000000,4.700000e+04,0.000000,9.440000,655.000000,0.000000,3.000000,3.000000,8.000000,15.000000,3.000000,27.000000,8596.000000,34.200000,5.123925e+04,30.700000,2.000000,43.000000,5939.250000,3058.000000,8000.000000,659.000000,0.000000,14800.000000,8.000000,8.000000,0.000000,679.000000,1.0
50%,13.00000,0.000000,13000.000000,14.000000,16500.000000,

This identifies values in the dataset that are spurious/misleading, like occasional ```dti``` entries of 9999. We remove such values by replacing them with the state average dti.

In [10]:
sampled_by_loan["addr_state"].unique()

array(['WA', 'IN', 'VA', 'KS', 'NC', 'SC', 'NY', 'AZ', 'PA', 'MD', 'RI',
       'IL', 'DC', 'SD', 'CO', 'NM', 'OH', 'FL', 'TX', 'GA', 'MA', 'CA',
       'MI', 'NJ', 'LA', 'TN', 'WV', 'AR', 'OR', 'CT', 'OK', 'NE', 'AL',
       'MO', 'WI', 'NH', 'MN', 'VT', 'NV', 'MT', 'WY', 'ME', 'AK', 'DE',
       'UT', 'KY', 'ND', 'MS', 'HI', 'ID'], dtype=object)

In [11]:
sampled_by_loan, num_vars_of_interest, cat_vars_of_interest = clean_accepted_df(sampled_by_loan, numeric_cols= num_vars_of_interest, categorical_cols= cat_vars_of_interest, one_hot_threshold=10)
sampled_by_loan.head()

,mths_since_rcnt_il,open_il_12m,loan_amnt,mths_since_recent_bc,total_bc_limit,total_il_high_credit_limit,pct_tl_nvr_dlq,total_cu_tl,mths_since_last_delinq,issue_d,mths_since_recent_bc_dlq,mths_since_recent_inq,fico_range_low,funded_amnt_inv,num_bc_tl,max_bal_bc,last_credit_pull_d,dti,total_bal_ex_mort,installment,num_op_rev_tl,percent_bc_gt_75,il_util,bc_open_to_buy,num_accts_ever_120_pd,delinq_2yrs,open_rv_24m,mths_since_last_record,num_bc_sats,mths_since_recent_revol_delinq,inq_last_12m,acc_open_past_24mths,annual_inc,open_rv_12m,int_rate,last_fico_range_low,inq_fi,num_actv_rev_tl,num_rev_tl_bal_gt_0,open_acc,total_acc,num_il_tl,mths_since_last_major_derog,total_bal_il,bc_util,tot_hi_cred_lim,revol_util,num_actv_bc_tl,all_util,revol_bal,avg_cur_bal,funded_amnt,last_fico_range_high,open_il_24m,total_rev_hi_lim,num_sats,num_rev_accts,inq_last_6mths,fico_range_high,sub_grade,emp_title,emp_length,purpose,desc,addr_state,lat,long,term__36_months,term__60_months,grade_a,grade_b,grade_c,grade_d,grade_e,grade_f,grade_g,home_ownership_any,home_ownership_mortgage,home_ownership_none,home_ownership_own,home_ownership_rent,verification_status_not_verified,verification_status_source_verified,verification_status_verified,initial_list_status_f,initial_list_status_w,policy_code_1.0,application_type_individual,application_type_joint_app,hardship_flag_n,hardship_flag_y,hardship_type_interest_only-3_months_deferral,hardship_type_no_hardship,hardship_reason_disability,hardship_reason_divorce,hardship_reason_excessive_obligations,hardship_reason_family_death,hardship_reason_income_curtailment,hardship_reason_medical,hardship_reason_natural_disaster,hardship_reason_no_hardship,hardship_reason_reduced_hours,hardship_reason_unemployment,loan_status_charged_off,loan_status_current,loan_status_fully_paid,loan_status_in_grace_period,loan_status_late_(16-30_days),loan_status_late_(31-120_days)
0,9.0,1.0,12000.0,9.0,24100.0,120544.0,87.2,1.0,33.0,699.0,69.0,15.0,690.0,12000.0,6.0,12597.0,1247.0,24.04,103281.0,391.62,8.0,66.7,66.0,3884.0,4.0,0.0,2.0,61.0,3.0,69.0,0.0,5.0,98000.0,1.0,10.78,750.0,1.0,5.0,5.0,15.0,39.0,17.0,69.0,82819.0,83.9,478544.0,62.8,3.0,65.0,20462.0,26756.0,12000.0,754.0,3.0,32600.0,15.0,14.0,0.0,694.0,8,10669,306,2,No Description,4,46.4659,-118.7344,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
1,19.0,0.0,8650.0,8.0,2750.0,18248.0,100.0,0.0,1600.0,699.0,1350.0,0.0,675.0,8650.0,2.0,653.0,882.0,25.49,18926.0,320.99,17.0,50.0,51.0,1375.0,0.0,0.0,16.0,30.0,2.0,1600.0,5.0,17.0,55000.0,12.0,19.89,635.0,2.0,17.0,13.0,18.0,19.0,2.0,1830.0,9358.0,45.0,38998.0,46.0,2.0,46.0,9568.0,1051.0,8650.0,639.0,1.0,20750.0,18.0,17.0,4.0,679.0,22,10470,306,2,No Description,31,39.7757,-86.1096,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
2,13.0,0.0,25000.0,13.0,38400.0,82117.0,100.0,0.0,1600.0,699.0,1350.0,0.0,745.0,25000.0,6.0,8937.0,1186.0,26.02,68056.0,777.55,5.0,20.0,58.0,17538.0,0.0,0.0,1.0,1200.0,5.0,1600.0,1.0,2.0,109000.0,0.0,7.49,720.0,1.0,3.0,3.0,9.0,19.0,7.0,1830.0,47194.0,54.3,373572.0,54.3,3.0,57.0,20862.0,33976.0,25000.0,724.0,1.0,38400.0,9.0,9.0,1.0,749.0,3,11902,306,2,No Description,19,39.1682,-78.1693,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
3,47.0,0.0,4225.0,26.0,2600.0,17522.0,66.7,0.0,18.0,699.0,1350.0,14.0,725.0,4225.0,3.0,367.0,1430.0,15.22,4888.0,146.16,5.0,0.0,22.0,2233.0,0.0,2.0,0.0,1200.0,3.0,18.0,0.0,0.0,35000.0,0.0,14.85,0.0,0.0,2.0,2.0,6.0,6.0,1.0,1830.0,3830.0,14.1,21822.0,24.6,1.0,22.0,1058.0,815.0,4225.0,499.0,0.0,4300.0,6.0,5.0,0.0,729.0,14,16571,306,2,No Description,23,37.6923,-97.3374,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,2.0,1.0,1400.0,28.0,34500.0,55501.0,100.0,2.0,1600.0,699.0,1350.0,18.0,700.0,1400.0,11.0,7386.0,1704.0,34.95,75258.0,47.10,12.0,75.0,67.0,7940.0,0.0,0.0,2.0,1200.0,8.0,1600.0,0.0,4.0,64000.0,0.0,12.88,700.0,0.0,11.0,11.0,17.0,24.0,3.0,1830

In [12]:
print(f"Number of missing values: {sampled_by_loan.isna().sum().sum()}")

Number of missing values: 0


In [14]:
sampled_by_loan[num_vars_of_interest].describe()

,mths_since_rcnt_il,open_il_12m,mths_since_recent_bc,total_bc_limit,total_il_high_credit_limit,pct_tl_nvr_dlq,total_cu_tl,mths_since_last_delinq,issue_d,mths_since_recent_bc_dlq,mths_since_recent_inq,funded_amnt_inv,fico_range_low,num_bc_tl,inq_last_6mths,max_bal_bc,last_credit_pull_d,fico_range_high,dti,total_bal_ex_mort,installment,num_op_rev_tl,percent_bc_gt_75,il_util,bc_open_to_buy,num_accts_ever_120_pd,delinq_2yrs,open_rv_24m,mths_since_last_record,num_bc_sats,mths_since_recent_revol_delinq,inq_last_12m,acc_open_past_24mths,annual_inc,open_rv_12m,int_rate,last_fico_range_low,inq_fi,num_actv_rev_tl,num_rev_tl_bal_gt_0,total_acc,num_il_tl,mths_since_last_major_derog,total_bal_il,bc_util,tot_hi_cred_lim,revol_util,num_actv_bc_tl,all_util,revol_bal,avg_cur_bal,funded_amnt,last_fico_range_high,open_il_24m,total_rev_hi_lim,num_sats,num_rev_accts,open_acc,loan_amnt
count,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,4.062200e+04,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,4.062200e+04,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000,40622.000000
mean,1232.689282,0.466299,86.314534,23684.479272,44444.088376,93.989161,1.040914,824.531190,999.988676,1036.841145,32.485944,15181.559223,698.315568,7.661612,0.550416,5391.727438,1754.426518,702.315716,18.789155,51733.410763,448.875696,8.341662,41.007932,70.288440,11789.013663,0.518143,0.320885,2.554699,1008.151740,4.809463,1066.242258,1.720866,4.609670,7.832073e+04,1.214440,13.016267,676.527867,0.687214,5.663335,5.602211,24.288169,8.474423,1342.063611,31714.300625,56.892795,1.781516e+05,49.515878,3.684161,57.217493,16795.099035,13360.626828,15186.276525,688.375265,1.380779,35288.389050,11.739304,14.043548,11.783393,15186.276525
std,1711.284217,0.830735,602.750744,23931.433835,45442.334742,9.187799,2.363655,782.881437,498.590707,558.946553,72.956016,9288.291414,33.073082,4.732256,0.850023,4946.558428,257.152101,33.073739,9.255615,49814.550767,270.441946,4.843187,35.967615,18.350381,17368.851971,1.367721,0.886263,2.233617,423.998349,3.140370,741.773874,2.048944,3.223653,5.949613e+04,1.280553,4.874614,109.314856,1.313097,3.453474,3.344764,12.215184,7.410487,795.836637,36883.192680,28.481930,1.785255e+05,24.742099,2.355986,17.415862,22423.222574,16230.889993,9290.192340,71.797607,1.324674,34974.412526,5.807124,8.229270,5.822492,9290.192340
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,750.000000,660.000000,0.000000,0.000000,0.000000,151.000000,664.000000,0.000000,0.000000,30.120000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,5.310000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,1000.000000
25%,10.000000,0.000000,6.000000,8400.000000,15145.750000,91.000000,0.000000,32.000000,608.000000,1350.000000,2.000000,8000.000000,675.000000,4.000000,0.000000,3209.000000,1734.000000,679.000000,12.100000,21075.000000,252.880000,5.000000,0.000000,68.000000,1897.000000,0.000000,0.000000,1.000000,1200.000000,3.000000,49.000000,1.000000,2.000000,4.700000e+04,0.000000,9.440000,655.000000,0.000000,3.000000,3.000000,15.000000,3.000000,74.000000,14903.250000,34.500000,5.123925e+04,30.700000,2.000000,50.000000,5939.250000,3058.000000,8000

### Exploratory Data Analysis